# Markov Model Sequence Generator

Use python (+ libs) to create random fastq reads that look like real reads.

In [63]:
import numpy as np
from scipy.stats import itemfreq
from __future__ import division

First, define a basic function to generate random sequence based on initial probabilities and transition probabilities:

In [64]:
def gen_seq(t_mat, initial_probs, seq_len):
    """Uses a Markov model to generate a DNA sequence"""
    nucleotides = np.array(['A', 'C', 'G', 'T'])
    seq = []
    first_nt = np.random.choice(nucleotides, 1, replace=True, p=initial_probs)
    seq.append(first_nt[0])
    for i in xrange(1, seq_len):
        prev_nuc = seq[i-1]
        prev_pos = np.where(nucleotides==prev_nuc)
        probs = np.array(t_mat[prev_pos]).ravel()
        nt = np.random.choice(nucleotides, 1,replace=True, p=probs)
        seq.append(nt[0])
    return seq

In order to base our probabilities on real data, we need to read fasta/q files. We can borrow Heng Li's function from https://github.com/lh3/readfq

In [65]:
# Heng Li's readfq function
def readfq(fp): # this is a generator function
    last = None # this is a buffer keeping the last unprocessed line
    while True: # mimic closure; is it a bad idea?
        if not last: # the first record or a record following a fastq
            for l in fp: # search for the start of the next record
                if l[0] in '>@': # fasta/q header line
                    last = l[:-1] # save this line
                    break
        if not last: break
        name, seqs, last = last[1:].partition(" ")[0], [], None
        for l in fp: # read the sequence
            if l[0] in '@+>':
                last = l[:-1]
                break
            seqs.append(l[:-1])
        if not last or last[0] != '+': # this is a fasta record
            yield name, ''.join(seqs), None # yield a fasta record
            if not last: break
        else: # this is a fastq record
            seq, leng, seqs = ''.join(seqs), 0, []
            for l in fp: # read the quality
                seqs.append(l[:-1])
                leng += len(l) - 1
                if leng >= len(seq): # have read enough quality
                    last = None
                    yield name, seq, ''.join(seqs); # yield a fastq record
                    break
            if last: # reach EOF before reading enough quality
                yield name, seq, None # yield a fasta record instead
                break

Now that we have our functions defined, let's test out the Markov model with arbitrary probabilities.

In [66]:
t_mat = np.matrix([[0.2, 0.3, 0.3, 0.2],[0.4,0.1,0.1,0.4],[0.6,0.1,0.1,0.2],[0.3,0.2,0.2,0.3]])
initial_probs = np.array([0.2,0.3,0.3,0.2])
seq_len=500
b = gen_seq(t_mat, initial_probs, seq_len)
print ''.join(b)

CATCTGATGACTTGTTAGAGATAGATATAACACTGTAGGTAGAGTGTTCCACATCAAATGAGCTCGATTTAGAGCACACACTTGAATTCTACATCTCACTAGAGGTAGACTGCACAGACATCAGAGGAGATTACTTACAGTATATACTGTATTCTCTAGAATTAGATTCTGAAACAGGAACTGTGTTTAGATCTTAGACTAGACACGTGTAGATTGCTAGATACTCACTCTTCTAAATTCTAGGTTTACAGGATAAGACACTTCACCTACTTATACGACAGATCTATGGTTACACTATTCACAAGTTTCTTTTCGAACATAGAATATCATACAAACACAAGGCTCTACACTACTCACATGTCCACTTGTTTACTGATCTTACATAAATACTTAAGTGAAGAACCATAATCACCGAGAGACTGACTACGTTTAACTATGCTGATAATAAATGAGAACTTTTCGACAGACGGAATCTAGTTTAGAGACTCAGTAGCACCATT


Now we can write some code to grab the probabilities from real data. First we load the file and store it in a numpy array.

In [67]:
# initial prob should come from frequencies of first nucleotide spot in all reads
# transition prob should come from data:

infile="/Users/arundurvasula/Downloads/55_ACAGTG_L001_R1_001.fastq.gz"
subfile="/Users/arundurvasula/Downloads/sub1.fastq"
reads = []

#for name, seq, qual in readfq(open(subfile)): 
#    reads.append(list(seq))
for name, seq, qual in readfq(open(subfile)): 
    reads.append(list(seq))    
    
read_mat = np.array(reads)

Then we get the initial probabilities by looking at the frequencies on nucleotides in the first position of the reads.

In [68]:
# to get initial_probs
first_pos = np.array(read_mat[:,0])
unique, counts = np.unique(np.reshape(first_pos, len(first_pos)), return_counts=True)
total = sum(counts)
initial_probs = [x for x in counts/total]
print initial_probs

[0.13550000000000001, 0.49990000000000001, 0.28089999999999998, 0.083699999999999997]


And the transition probabilities can come from the frequency of the rest of the data.

In [69]:
afterA = np.array([0,0,0,0])
afterC = np.array([0,0,0,0])
afterG = np.array([0,0,0,0])
afterT = np.array([0,0,0,0])
i = 0
oldval=""
currval=""
for (x,y), nuc in np.ndenumerate(read_mat):
    if i > 1:
        currval=nuc
        if oldval == "A" and currval == "A":
            afterA[0] = afterA[0] + 1
        elif oldval == "A" and currval == "C":
            afterA[1] = afterA[1] + 1
        elif oldval == "A" and currval == "G":
            afterA[2] = afterA[2] + 1
        elif oldval == "A" and currval == "T":
            afterA[3] = afterA[3] + 1
        #----
        elif oldval == "C" and currval == "A":
            afterC[0] = afterC[0] + 1
        elif oldval == "C" and currval == "C":
            afterC[1] = afterC[1] + 1
        elif oldval == "C" and currval == "G":
            afterC[2] = afterC[2] + 1
        elif oldval == "C" and currval == "T":
            afterC[3] = afterC[3] + 1
        #----
        elif oldval == "G" and currval == "A":
            afterG[0] = afterG[0] + 1
        elif oldval == "G" and currval == "C":
            afterG[1] = afterG[1] + 1
        elif oldval == "G" and currval == "G":
            afterG[2] = afterG[2] + 1
        elif oldval == "G" and currval == "T":
            afterG[3] = afterG[3] + 1
        #----
        elif oldval == "T" and currval == "A":
            afterT[0] = afterT[0] + 1
        elif oldval == "T" and currval == "C":
            afterT[1] = afterT[1] + 1
        elif oldval == "T" and currval == "G":
            afterT[2] = afterT[2] + 1
        elif oldval == "T" and currval == "T":
            afterT[3] = afterT[3] + 1
        oldval = nuc
    i = i+1

totalA = sum(afterA)
afterA_p = [x for x in afterA/totalA]
totalC = sum(afterC)
afterC_p = [x for x in afterC/totalC]
totalG = sum(afterG)
afterG_p = [x for x in afterG/totalG]
totalT = sum(afterT)
afterT_p = [x for x in afterT/totalT]
t_mat = np.matrix([afterA_p,afterC_p,afterG_p,afterT_p])

print t_mat

[[ 0.32096067  0.19746804  0.22908493  0.25248636]
 [ 0.3116406   0.26451757  0.13073695  0.29310488]
 [ 0.28586095  0.23622122  0.25115496  0.22676287]
 [ 0.19229158  0.26865551  0.23050294  0.30854997]]


Now that we have that, let's see what an example read would look like.

In [76]:
seq_len=50
b = gen_seq(t_mat, initial_probs, seq_len)
print ''.join(b)

CTGGAAGTCATCTTCGTTCGGAGGGGGGTTTGTTTGTAGCCTAGCCACAT


And finally, we can generate a file of simulated reads.

In [ ]:
a = open("/Users/arundurvasula/Downloads/simreads.fasta", 'w')
for i in xrange(0, 500000):
    b = gen_seq(t_mat, initial_probs, seq_len)
    c = ''.join(b)
    a.write(">"+str(i)+'\n')
    a.write(c+'\n')